# Actividad: Problemas de búsqueda

- Diseño de agentes inteligentes
- Profesor:
- Equipo:

| Nombre | Matrícula |
| ----- | ---- |
| Juan Pablo Echeagaray González | A0083646 |
| Emily Rebeca Méndez Cruz | A00830768 |

In [88]:
import numpy as np
from simpleai.search import SearchProblem
from simpleai.search import astar


## Problema 1

El problema de los misioneros y los caníbales usualmente se describe de la siguiente manera. Tres misioneros y tres caníbales están en un lado de un río, a través del cual se puede cruzar con un bote que puede llevar a una o dos personas. El problema es encontrar una manera de que crucen las seis personas, de tal forma que en ningún momento el número de caníbales sobrepase el número de misioneros en alguno de los lados del río, si en dicho lado hay misioneros.

1. Formulen el problema de búsqueda, indicando el espacio de estados y los demás elementos de un problema.

En el problema tenemos dos lados, el izquierdo donde se encuentran los tres misioneros y los tres caníbales y al bote al inicio del problema, y el lado derecho donde debemos llevar a los misioneros y caníbales como objetivo final, tomando en cuenta las condiciones del problema.
Tenemos que (M,C,B) donde:
- M son los misioneros
- C son los caníbales
- B es el bote
por lo que el estado inicial es (3,3,1) del lado izquierdo y queremos llegar a ser (0,0,0).

Generamos los estados que tiene el problema:

![](img\Problemas_de_búsqueda.jpg)

Tenemos que los cuadros verdes son los estados que se pueden realizar cumpliendo la condición establecida, los rojos son los estados donde los misioneros son comidos y los blancos imposibles, ya que no se puede iniciar sin bote y terminar con el bote del lado izquierdo.

2. Resuelvan el problema a mano utilizando búsqueda de anchura.

De nuestras opciones tenemos que pueden ir en el bote:
- Un misionero
- Un canibal
- Dos misioneros
- Dos caníbales
- Un misionero y un caníbal

Con los estados generados anteriormente haremos un diagrama de árbol revisando que se cumpla la siguiente condición:
Que el número de caníbales no sobrepase el número de misioneros en ningún momento ni en alguno de los lados del río.
Gracias al esquema anterior podemos decir que los estados:
- 2,3,1
- 1,3,0
- 2,3,0
- 1,3,1
- 2,1,1
- 2,0,1
- 2,1,0
- 2,0,0
- 1,0,1
- 1,0,0
- 1,2,1
- 1,2,0

son los estados en donde los misioneros son comidos, por lo que no los tomaremos en cuenta para llegar a la meta.

![](img\Problemas_de_búsqueda_(1).jpg)

Los estados grises son los estados repetidos, no escogeremos estos debido a que si lo hiciéramos esto generaría un ciclo y no llegaríamos al objetivo del problema.

Con el Breadth first search (BFS) o búsqueda de anchura buscamos desde la raíz y continuamos con todos los nodos vecinos de este, revisamos que se cumpla la condición dada en cada nodo y que no nos mande a un loop y con esto trazar un camino a la meta establecida o en este caso que todas las personas y el bote queden del lado derecho del río.

![](img\Problemas_de_búsqueda_(3).jpg)

En conclusión con el BFS pasaremos 12 estados, incluyendo el estado inicial y final, para completar el problema establecido.

3. Contesten lo siguiente:

- ¿Consideran que es una buena idea verificar estados repetidos para realizar la búsqueda?

Sí, debido a que si se escogiera un estado repetido dentro del problema se generaría un loop dentro de la búsqueda, ya que escogiendo este volveríamos a regresar al mismo estado y así sucesivamente.


## Problema 2

## Problema 3

## Problema 4: Rompecabezas deslizante

## Problema 5: Laberintos

Utilicen el algoritmo A* para encontrar un camino entre los dos puntos indicados en el siguiente laberinto.

```txt
++++++++++++++++++++++
+ O +   ++ ++        +
+     +     +++++++ ++
+ +    ++  ++++ +++ ++
+ +   + + ++         +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +     X +
++++++++++++++++++++++
```

- El símbolo + indica un obstáculo o pared. El símbolo O representa el punto de inicio, y X indica el objetivo.
- Inventen otros tres laberintos, y prueben de nuevo el algoritmo A*.

> Nota: Usen como heurística la distancia entre el punto actual y el punto objetivo.

In [89]:
maze = '''
++++++++++++++++++++++
+ O +   ++ ++        +
+     +     +++++++ ++
+ +    ++  ++++ +++ ++
+ +   + + ++         +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +     X +
++++++++++++++++++++++
'''

maze_2 = '''
++++++++++++++++++++++
+   +   ++O++        +
+     +     +++++++ ++
+ +    ++  ++++ +++ ++
+ +   + + ++         +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+X         + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++
'''

maze_3 = '''
++++++++++++++++++++++
+ O +   ++ ++        +
+     +     +++++++ ++
+ +        ++++ +++ ++
+ +   + +            +
+          ++  ++  + +
+++++++ +  ++  ++  + +
+++++ ++++++ +  ++   +
+          + +  + +  +
+++++ + X            +
++++++++++++++++++++++
'''

maze_4 = '''
++++++++++++++++++++++
+ O +   ++ ++        +
++    +     +++++++ ++
+++    ++    ++ +++ ++
+++++++ + ++         +
+X         ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++
'''


In [90]:
COSTS = {
    'up': 1,
    'down': 1,
    'left': 1,
    'right': 1,
}

In [91]:
class Labyrinth(SearchProblem):
    def __init__(self, maze):
        """Constructor for the labyrinth game.

        Args:
            - maze (list[list]): Array representation of the labyrinth
        """        
        self.board = maze

        for y in range(len(self.board)):
            for x in range(len(self.board[y])):
                if self.board[y][x].lower() == 'x':
                    self.goal = (x, y)
                elif self.board[y][x].lower() == 'o':
                    self.start = (x, y)
        super(Labyrinth, self).__init__(initial_state=self.start)

    
    def actions(self, state):
        """Returns a list of all possible actions from the current state.

        Args:
            - state (tuple(int)): x, y coordinates of the current position

        Returns:
            - list[str]: list of all possible actions from the current state
        """        
        actions = []

        for action in list(COSTS.keys()):
            x, y = self.result(state, action)
            if self.board[y][x] != '+':
                actions.append(action)

        return actions

    
    def result(self, state, action):
        """Result function for the labyrinth game.

        Args:
            - state (tuple(int)): x, y coordinates of the current position
            - action (str): Movement to be made

        Returns:
            - tuple(int): New position coordinates
        """        
        x, y = state
        if action.count('up'):
            y -= 1
        if action.count('down'):
            y += 1
        if action.count('left'):
            x -= 1
        if action.count('right'):
            x += 1

        new_state = (x, y)
        return new_state


    def is_goal(self, state):
        """Check if the current state is the goal state.

        Args:
            - state (tuple(int)): x, y coordinates of the current position

        Returns:
            - bool: whether the current state is the goal state
        """        
        return state == self.goal


    def cost(self, state, action, state2):
        """Returns cost of the movement.

        Args:
            - state (tuple(int)): x, y coordinates of the current position
            - action (str): movement to be made
            - state2 (tuple(int)): x, y coordinates of the new position

        Returns:
            - float: cost of the movement
        """        
        return COSTS[action]


    def heuristic(self, state):
        """Heuristic function for the labyrinth game.

        Args:
            - state (tuple(int)): x, y coordinates of the current position

        Returns:
            - float: Euclidean distance to the goal state
        """        
        x, y = state
        xg, yg = self.goal
        return np.sqrt((x - xg)**2 + (y - yg)**2)
    

In [92]:
def solver(maze):
    """Maze solver.

    Args:
        - maze (list[list]): Array representation of the labyrinth
    """    
    MAZE = [list(x) for x in maze.split('\n') if x]
    problem = Labyrinth(MAZE)
    result = astar(problem, graph_search=True)
    
    path = [x[1] for x in result.path()]

    for y in range(len(MAZE)):
        for x in range(len(MAZE[y])):
            if (x, y) == problem.start:
                print("o", end='')
            elif (x, y) == problem.goal:
                print("x", end='')
            elif (x, y) in path:
                print(".", end='')
            else:
                print(MAZE[y][x], end='')
        print()


In [93]:
print('Laberinto 1')
solver(maze)

Laberinto 1
++++++++++++++++++++++
+ o.+   ++ ++        +
+  ...+     +++++++ ++
+ +  . ++  ++++ +++ ++
+ +  .+ + ++         +
+    ......++  ++  + +
+++++ + + .....++  + +
+++++ +++  + +..++   +
+          + + .+ +  +
+++++ +  + + + ....x +
++++++++++++++++++++++


In [94]:
print('Laberinto 2')
solver(maze_2)

Laberinto 2
++++++++++++++++++++++
+   +   ++o++        +
+     +  .. +++++++ ++
+ +    ++. ++++ +++ ++
+ +   + +.++         +
+    ..... ++  ++  + +
+++++.+ +      ++  + +
+++++.+++  + +  ++   +
+x....     + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++


In [95]:
solver(maze_3)

++++++++++++++++++++++
+ o +   ++ ++        +
+ ..  +     +++++++ ++
+ +....... ++++ +++ ++
+ +   + +.....       +
+          ++. ++  + +
+++++++ +  ++..++  + +
+++++ ++++++ +. ++   +
+          + +. + +  +
+++++ + x......      +
++++++++++++++++++++++


In [96]:
solver(maze_4)

++++++++++++++++++++++
+ o +...++ ++        +
++....+...  +++++++ ++
+++    ++.   ++ +++ ++
+++++++ +.++         +
+x........ ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + +  + +  +
+++++ +  + + +       +
++++++++++++++++++++++
